In [53]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
from getcpuinfo import cpuinfo
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [30]:
def getTrainedLSTMModel():
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,50,1), return_sequences = True))
    
    model.add(Dense(50, activation = 'tanh'))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM(25,return_sequences = False))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("LSTMBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("LSTMBestWeights.hdf5")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;
    
    
def getTrained3OuptutModel():
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Flatten(batch_input_shape = (None,50,1)))
    
    model.add(Dense(50, activation = "relu"))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(25, activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Dense(12,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(6,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("3OutputDNNBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("3OutputDNNBestWeights.hdf5")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;

In [31]:
def appendToList(oldList,newList,maxLength):
    
    #append to the new entries
    oldList = np.append(oldList,np.array([newList]),axis = 0)
    
    #Remove oldest row if there are more than 50 entries
    if(oldList.shape[0]>maxLength):
        oldList = oldList[1:,:]

    return oldList

def getPredictionsForOneTimeStep(model,usageList,maxLengthParam):
    
    currUsageList = cpuinfo()
    
    updatedList = appendToList(usageList,currUsageList,maxLengthParam)
    
    predictions = []
    
    for i in range(updatedList.shape[1]):
        
        if updatedList.shape[0]==maxLengthParam:
            
            modelOutput = model.predict(np.expand_dims(updatedList[i],axis=0))

            modelPrediction = np.argmax(modelOutput)

            predictions.append(modelPrediction)
        
        else:
            
            predictions.append(0)
    
    return predictions, updatedList

#overProvisionedList = 

def getRightOutputs(usageList,maxLengthParam):
    
    currList = usageList[usageList.shape[0]-1,:]
    
    target = []
    
    for i in range(usageList.shape[1]):
        if currList[i] > maxThreshold:# Scale Up
            target.append(1)
        elif currList[i] < minThreshold: # Scale Down
            target.append(-1)
        else:
            target.append(0)  #Remain same
    
    if(length(target)>maxLengthParam):
        
        target = target[1:]
    
    return target

def getPredictions(model,usageList, maxLengthParam):
    
    predictions,updatedList = getPredictionsForOneTimeStep(model,usageList,maxLengthParam)
    
    correctOutputs = getRightOutputs(updatedList)
    
    correctOutputsForTraining = to_categorical(correctOutputs,num_classes = 3)
    
    

In [54]:
mylist = np.array([cpuinfo()])
print(mylist)
print(mylist.shape)

[[16.9762474  9.5147787  0.        65.0693186  0.0819067]]
(1, 5)


In [ ]:
mylist = np.append(np.array([[]]), np.array([cpuinfo()]), axis = 0)
print(mylist)
print(mylist.shape)

In [57]:
mylist[:,1]

array([9.5147787, 8.5266884])

In [58]:
a = [1,2,3,4]
a[1:]

[2, 3, 4]